This bot will being with equal positions of both a 3s and 3l leveraged token on kucoin \
When the position gains X% it will trigger a sell\
When a sell is triggered the resulting usdt will take profit then purchase back in immediately 50/50\
Every time it re-buys, update buy-in frame

In [421]:
from dotenv import load_dotenv
import os
from pathlib import Path
import requests
from requests import Request, Session
import datetime
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
import time
from kucoin.trade.trade import TradeData
from kucoin.client import Trade, User, Market
import base64
import hmac
import hashlib

In [422]:
load_dotenv();

In [423]:
api_key = os.getenv('KUCOIN_API_KEY')
api_secret = os.getenv('KUCOIN_SECRET_KEY')
api_passphrase = os.getenv('KUCOIN_PASSPHRASE')

In [424]:
Tclient = Trade(api_key, api_secret, api_passphrase)
Uclient = User(api_key, api_secret, api_passphrase)
Mclient = Market(api_key, api_secret, api_passphrase)

In [410]:
def kucoin_func(ticker):
    # Request data from api
    crypto = requests.get(f'https://api.kucoin.com/api/v1/market/orderbook/level1?symbol={ticker}-USDT')
    
    # Import api data as json    
    crypto_j = crypto.json()
    
    # Create pandas df from json file    
    crypto_df = pd.DataFrame(crypto_j['data'], index=[0])
    
    # Rename columns
    crypto_df = pd.DataFrame(crypto_df['bestBid']).rename(columns = {'bestBid':f'Bid {ticker}'})
    
    return crypto_df

In [411]:
#List of dictionaries for long and short tokens
leverage_tokens = {'long_tokens':['BTC3L','AAVE3L','ADA3L','ATOM3L','AVAX3L',
                                  'AXS3L','BCH3L','BNB3L','DOGE3L','DOT3L',
                                  'EOS3L','ETH3L','FTM3L','GALAX3L','LINK3L',
                                  'LTC3L','LUNA3L','MANA3L','MATIC3L','NEAR3L',
                                 'SAND3L','SOL3L','SUSHI3L','UNI3L','VET3L','XRP3L'
                                 ],
          'short_tokens':['BTC3S','AAVE3S','ADA3S','ATOM3S','AVAX3S',
                                  'AXS3S','BCH3S','BNB3S','DOGE3S','DOT3S',
                                  'EOS3S','ETH3S','FTM3S','GALAX3S','LINK3S',
                                  'LTC3S','LUNA3S','MANA3S','MATIC3S','NEAR3S',
                                 'SAND3S','SOL3S','SUSHI3S','UNI3S','VET3S','XRP3S']}

In [412]:
#Function While loop for retrieval of best bids
short_buy_in = pd.DataFrame({'BTC3S': [1.7383],
 'AAVE3S': [12.0174],
 'ADA3S': [1.3684],
 'ATOM3S': [1.3447],
 'AVAX3S': [2.1905],
 'AXS3S': [3.9006],
 'BCH3S': [6.7773],
 'BNB3S': [6.7283],
 'DOGE3S': [6.37],
 'DOT3S': [2.9218],
 'EOS3S': [3.0024],
 'ETH3S': [0.016099],
 'FTM3S': [0.3117],
 'GALAX3S': [23.7855],
 'LINK3S': [1.3273],
 'LTC3S': [1.4896],
 'LUNA3S': [0.020995],
 'MANA3S': [2.1292],
 'MATIC3S': [0.3443],
 'NEAR3S': [0.003693],
 'SAND3S': [0.070237],
 'SOL3S': [2.6776],
 'SUSHI3S': [9.3603],
 'UNI3S': [11.5251],
 'VET3S': [1.5435],
 'XRP3S': [26.8833]}, index=[0])

long_buy_in = pd.DataFrame({'BTC3L': [0.197],
 'AAVE3L': [0.7447],
 'ADA3L': [0.051602],
 'ATOM3L': [1.1956],
 'AVAX3L': [1.9968],
 'AXS3L': [1.3251],
 'BCH3L': [0.1033],
 'BNB3L': [3.7316],
 'DOGE3L': [0.2798],
 'DOT3L': [0.779],
 'EOS3L': [0.025661],
 'ETH3L': [0.581],
 'FTM3L': [1.012],
 'GALAX3L': [0.2524],
 'LINK3L': [2.663],
 'LTC3L': [0.2373],
 'LUNA3L': [5.2303],
 'MANA3L': [1.2569],
 'MATIC3L': [4.3482],
 'NEAR3L': [6.8523],
 'SAND3L': [7.5524],
 'SOL3L': [0.6442],
 'SUSHI3L': [0.3155],
 'UNI3L': [0.6071],
 'VET3L': [0.01042],
 'XRP3L': [0.1663]}, index=[0])

In [432]:
long_df = pd.DataFrame().astype(float)
short_df = pd.DataFrame().astype(float)
long_rename = pd.DataFrame()
short_rename = pd.DataFrame()
leverage_return = None
pos_returns = None

while leverage_return is None:
    try:
        #Loop
        for x in leverage_tokens:
    
# Run function for each token in list and add to func dictionary
            for y in leverage_tokens['short_tokens']:
     
                short_df[y] = pd.DataFrame(kucoin_func(y)).astype(float)
                short_sub = short_df.sub(short_buy_in)
                short_return = short_sub.divide(short_buy_in)
                short_rename[y[:-2]] = short_return[y]
    
            for z in leverage_tokens['long_tokens']:
                long_df[z] = pd.DataFrame(kucoin_func(z)).astype(float)
                long_sub = long_df.sub(long_buy_in)
                long_return = long_sub.divide(long_buy_in)
                long_rename[z[:-2]] = long_return[z]
    
            leverage_return = long_rename.add(short_rename, axis = 0).T
            leverage_return = (leverage_return/2)*100
            pos_return = leverage_return.loc[leverage_return[0] >= 1.2]
    except:
        pass

In [433]:
print(leverage_return)
print(pos_return)

               0
BTC    -0.258027
AAVE   -1.322102
ADA    -2.460761
ATOM    0.752421
AVAX   -2.067476
AXS    -2.225254
BCH     0.019465
BNB    -3.122078
DOGE    1.955469
DOT    -1.959401
EOS    -1.126450
ETH    -1.340675
FTM    12.748407
GALAX  -4.313926
LINK   -0.269946
LTC    -2.255332
LUNA    0.826038
MANA   -2.002074
MATIC  -7.377335
NEAR    1.097689
SAND   -0.066576
SOL     1.549708
SUSHI   0.949455
UNI    -1.329433
VET    -3.349950
XRP    -1.231488
              0
DOGE   1.955469
FTM   12.748407
SOL    1.549708


In [416]:
account = Uclient.get_account_list()
account_df = pd.DataFrame(account).set_index('currency')
account_df = account_df.loc[account_df['type']=='trade']
account_df = pd.DataFrame(account_df['balance'].astype(float), index = account_df.index)
account_df = account_df.loc[account_df['balance'] > 0]
account_dict = account_df.to_dict()
account_dict = account_dict['balance']

balance_vals = {}

balance_3s_df = pd.DataFrame().astype(float)
balance_3l_df = pd.DataFrame().astype(float)

for x in leverage_return.index:        
    if leverage_return.loc[x].values.astype(float) >= 1.2:
        x_long = x+'3L'
        x_short = x+'3S'
        x_long_account = str(account_dict[x_long]*.99)
        x_short_account = str(account_dict[x_short]*.99)
        #Tclient.create_market_order(x_short+'-USDT','sell', size = x_short_account)
        #Tclient.create_market_order(x_long+'-USDT','sell', size = x_long_account)